#### reference: https://towardsdatascience.com/basic-binary-sentiment-analysis-using-nltk-c94ba17ae386

In [1]:
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re
import json

In [13]:
import datetime
import pandas as pd
import json
from newsapi import NewsApiClient

In [8]:
def get_news(my_key, my_query,from_date,to_date):
    newsapi = NewsApiClient(api_key=my_key)

    all_articles = newsapi.get_everything(q=my_query,
                                          from_param=from_date,
                                          to=to_date,
                                          language='en',
                                          sort_by='relevancy',
                                          page=5)
    return all_articles

In [14]:
from_date = datetime.datetime.now()
to_date = from_date - datetime.timedelta(30)
from_date = from_date.strftime('%Y-%m-%d')
to_date = to_date.strftime('%Y-%m-%d')
my_query = 'coronavirus'
all_articles = get_news('',my_query,from_date,to_date)

datetime.datetime(2020, 1, 20, 15, 11, 38, 724384)

In [ ]:
with open(my_query + '_news.json','w') as fout:
    json.dump(all_articles,fout)

# Load the content from json file
with open (my_query + '_news.json','r') as fin:
    all_articles = json.load(fin)

In [38]:
def extract_content(article):
    res_str = ''
    for _key in ['title','description','content']:
        _str = article[_key]
        if _str is not None:
            if _str.find('…') > 0:
                _str = _str[:_str.find('…')]
            res_str += _str + '. '  
        else:
            continue
    return res_str

In [39]:
dict_article = dict()
for article in all_articles['articles']:
    yyyymmdd = pd.to_datetime(article['publishedAt']).strftime('%Y%m%d')
    if dict_article.get(yyyymmdd):
        dict_article[yyyymmdd].append(extract_content(article))
    else:
        dict_article[yyyymmdd] =[extract_content(article)]

In [40]:
content = ' '.join([i[0] for i in dict_article.values()])

In [15]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.text import Text
import string, re

In [41]:
content = content.replace('\r\n',' ')

In [42]:
def remove_punctuation(sentence):
    sentence = re.sub(r'[^\w\s]','',sentence)
    return sentence

In [43]:
# Sentence seperation
cleaned_sent =[remove_punctuation(sentence) for sentence in sent_tokenize(content)]

In [45]:
# Word extraction
speech_words = [word_tokenize(sentence) for sentence in cleaned_sent]

In [47]:
from nltk.corpus import stopwords

In [49]:
stop_words = list(set(stopwords.words('english')))

In [50]:
def remove_stopword(sentence):
    return [w for w in sentence if not w in stop_words]

In [52]:
filtered_words = [remove_stopword(s) for s in speech_words ]

In [54]:
tagged_words = [nltk.pos_tag(_word) for _word in filtered_words]